In [7]:
import pandas as pd
from llms.OpenAIAssistantChatbot import OpenAIAssistantChatbot
from threading import Thread, Lock
import time

In [8]:
# read in data
df_samples = pd.read_csv('../data/sample/train_sample_unlabeled.csv', sep=';')
df_samples

,goal,sdt-cluster,title,description,gpt_sector,gpt_sector_reason,source
0,"I would like to be committed to a cause (e.g.,...",autonomy,Volunteering in Zaragoza with People with Disa...,Activity description\n The Project is c...,Betreuung von pflegebedürftigen Personen,Working with people with disabilities.,EU
1,I would like to practice religious traditions.,autonomy,Volunteering for older people,How you can get involvedDo you enjoy being aro...,Betreuung von pflegebedürftigen Personen,Besuche und Begleitung älterer Menschen,ULF
2,"I would like to have an exciting, stimulating ...",autonomy,volunteer/bingo support,extendicare michener hill's recreation departm...,Betreuung von pflegebedürftigen Personen,Bingo support für Bewohner,VC
3,I would like to help others.,autonomy,hair salon assistant - sylvan lake,bethany riverview is seeking a dynamic volunte...,Betreuung von pflegebedürftigen Personen,Supporting senior residents in salon.,VC
4,I would like to achieve salvation.,autonomy,Stay Active Volunteer vol. 2,Activity description\n InCo-Molfetta ha...,Bildung,Englischunterricht und Workshops,EU
...,...,...,...,...,...,...,...
600,"I would like to be physically healthy, maintai...",relatedness,foster dog care,"some volunteers provide a short-term, temporar...","Umwelt, Natur, Tierschutz",Fosterhundebetreuung,VC
601,I would like to control others.,relatedness,For Volunteers from Italy: Educazione ambient...,Activity description\n Il centro visita...,"Umwelt, Natur, Tierschutz","Die Tätigkeit umfasst den Schutz der Natur, di...",EU
602,I would like to avoid conflict with others.,relatedness,SJ/ESC67 – Timber recycling,"Activity description\n Since 2002, the ...","Umwelt, Natur, Tierschutz",Recycling von Holz,EU
603,I would like to be better than others.,relatedness,Italy - promoting mountain communities,Activity description\n Organization ALA...,"Umwelt, Natur, Tierschutz",Outdoor activities in mountains,EU


In [9]:
# make copy and create empty fields
df_samples_labeled = df_samples.copy()
df_samples_labeled['label'] = None
df_samples_labeled['score'] = None
df_samples_labeled['reason'] = None

In [10]:
# create a Lock for the threads
DF_LOCK = Lock()

In [11]:
# function for getting response and save to df with Lock
def get_response(row, index):
    if row['label'] is not None:
        return
    
    chatbot = OpenAIAssistantChatbot("asst_R5NEGWNwvUh0J7dsY5C1MqLV") # % gpt
    message = f"Goal: {row['goal']}\n\nTask-Title: {row['title']}\n\nTask-Description: {row['description']}"
    response = chatbot.Run_JSON(message)
    score = response['score']
    label = response['label']
    reason = response['reason']
    
    # update row in df
    row['label'] = label
    row['score'] = score
    row['reason'] = reason
    with DF_LOCK:
        df_samples_labeled.loc[index] = row

In [ ]:
# create a counter because Api is limited to 200 Requests per Minute
counter = 0
threads = []

# save timestamps
global_start_time = time.time()
start_time = time.time()

# iterate over all rows and start a thread for each row
for index, row in df_samples_labeled.iterrows():
    # check if 190 requests are done and wait for 60 seconds
    if counter == 190:
        print("Done 190 requests")
        cur_time = time.time()
        if cur_time - start_time < 61:
            time.sleep(60 - (cur_time - start_time))
        
        counter = 0
        start_time = time.time()
    # start a Thread for receiving the response from chatgpt 
    thread = Thread(target=get_response, args=(row, index))
    threads.append(thread)
    thread.start()
    counter += 1

# wait for all threads to finish
for thread in threads:
    thread.join()
    threads = []
    
print(f"Took about {time.time() - global_start_time} seconds to complete all requests")    

In [ ]:
# save labeled data
df_samples_labeled.to_csv('../data/labeled_data/gpt_labeled_train_sample.csv', sep=';', index=False)